In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os.path
from PIL import Image
import itertools
import scipy.io as sio

import matplotlib.pyplot as plt

In [12]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv1D
from tensorflow.keras.layers import MaxPooling1D, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler
from tensorflow.keras import backend as K

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
base_dir = 'data/'
data_dir = base_dir+'data_speech_commands_v0.02/'
 
train_txt = base_dir+'train_16words.txt'
val_txt = base_dir+'validation_16words.txt'
test_txt = base_dir+'test_16words.txt'

train_data = np.load('/users/jhlee/data/img_snd/train_data.npz')
test_data = np.load('/users/jhlee/data/img_snd/test_data.npz')

In [ ]:
X_train = train_data['X_train']
y_train = train_data['y_train']
X_test = test_data['X_test']
y_test = test_data['y_test']

In [ ]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

In [ ]:
img_dim = (99, 257)
img_dim_crop = (99, 257)

In [ ]:
def batch_generator(X, y, batch_size=64, 
                    shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [19]:
def build_cnn(conv_num=1, fcn_num=1):
    model=Sequential()
    model.add(Conv1D (kernel_size=3, filters=128, strides=3, padding='valid', 
                  activation='relu', input_shape=(16000, 1)))  
#     model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=4, strides=4, padding='same'))
    
    for i in range(conv_num-1):
        model.add(Conv1D (kernel_size=3, filters=128, strides=3, padding='valid', 
                          activation='relu'))  
#         model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=4, strides=4, padding='same'))

    model.add(Flatten())
    
    for i in range(fcn_num):
        model.add(Dense( 1024/(i+1), activation='relu' ))
        model.add(Dropout(0.5))
    
    model.add(Dense(16, activation='softmax' ))
    return model

In [20]:
model = build_cnn(conv_num=3, fcn_num=1)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 5333, 128)         512       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 1334, 128)         0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 444, 128)          49280     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 111, 128)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 37, 128)           49280     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 10, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1280)              0         
__________

In [ ]:
## Loss Function and Optimization
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
model_path = '/users/lww/data/checkpoint/AlexNet_based_check_point/'
os.makedirs(model_path, exist_ok=True)
model_filename = model_path+'{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", 
                               verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=100)